In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit

In [3]:
SIGMAY_PEAK = 0.0045
LAMBDA_ECC = np.array([500,510,520,490,530,526,480,470,515,460]) # ORDINE NECESSARIO
filename = 'data/ecc_rodamina_05.csv'
data = pd.read_csv(filename, sep=',', header=1,nrows=151)
data = data.drop(columns=['Unnamed: 20'])
data.columns = [f'λ {LAMBDA_ECC[i]}' if j % 2 == 0 else f'I {LAMBDA_ECC[i]}' for i in range(len(LAMBDA_ECC)) for j in range(2)]
# Drop unnamed
data.tail()

LAMBDA_ECC = np.sort(LAMBDA_ECC)

In [4]:
fig = go.Figure()
colors = px.colors.sequential.Viridis[:len(LAMBDA_ECC)]
for g,i in enumerate(LAMBDA_ECC):
    λ = data[f'λ {i}'].dropna().values
    I = data[f'I {i}'].dropna().values
    # c = colors[i]
    fig.add_trace(go.Scatter(x=λ, y=I, mode='lines', name=f'λ {i}', line=dict(color=colors[g])))

fig.update_layout(
    #title='Spettro di emissione della rodamina B con diversi lambda eccitazione',
    xaxis_title='λ [nm]',
    yaxis_title='I [a.u.]',
    legend_title='λ exc [nm]',
    width = 800,
    height = 600, 
    xaxis=dict(range=[540, 600])
    ,
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top'
    )
)

fig.write_html(dir_path + '/html/spettro_emissione_rodamina_eccitata_a_diverse_frequenze.html')
fig.write_image(dir_path + '/images/spettro_emissione_rodamina_eccitata_a_diverse_frequenze.png')
fig.show()

In [5]:
# COLLASSO SPETTRI:

fig = go.Figure()
for g,i in enumerate(LAMBDA_ECC):
    λ = data[f'λ {i}'].dropna().values
    I = data[f'I {i}'].dropna().values
    Imax = np.max(I)
    I = I/Imax
    # c = colors[i]
    fig.add_trace(go.Scatter(x=λ, y=I, mode='lines', name=f'λ {i}', line=dict(color=colors[g])))
fig.update_layout(
    #title='Spettro di emissione della rodamina B con diversi lambda eccitazione',
    xaxis_title='λ [nm]',
    yaxis_title='I/Imax',
    legend_title='λ exc [nm]',
    width = 800,
    height = 600, 
    xaxis=dict(range=[540, 600])
    ,
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top'
    )
)
fig.write_html(dir_path + '/html/spettro_emissione_rodamina_eccitata_a_diverse_frequenze_collasso.html')
fig.write_image(dir_path + '/images/spettro_emissione_rodamina_eccitata_a_diverse_frequenze_collasso.png')
fig.show()

In [6]:
def fit_picchi(x, a,x0,p):
    return -a * (x-x0)**2 + p



Picchi = []
sigma_PICCHI = []

for i in LAMBDA_ECC:
    λ = data[f'λ {i}'].dropna().values
    I = data[f'I {i}'].dropna().values

    idλmax = np.argmax(I)
    Imax = I[idλmax]
    λc = λ[idλmax]

    λ = λ[idλmax-5:idλmax+5]
    I = I[idλmax-5:idλmax+5]
    sigmaI = I * SIGMAY_PEAK

    # Fit picco
    popt, pcov = curve_fit(fit_picchi, λ,I, p0=[1, λc, Imax], sigma=sigmaI, absolute_sigma=True)
    a, x0, p = popt
    err_a, err_x0, err_p = np.sqrt(np.diag(pcov))
    
    Picchi.append(p)
    sigma_PICCHI.append(err_p)

Picchi = np.array(Picchi)
sigma_PICCHI = np.array(sigma_PICCHI)

print(f'Picchi: {Picchi}')
print(f'Errori picchi: {sigma_PICCHI}')

Picchi: [110.0832379  163.88926662 280.30319838 393.06505053 466.18974343
 639.88802432 758.75275388 864.53429603 960.11318261 923.06582293]
Errori picchi: [0.23426658 0.33190078 0.59480244 0.787656   0.98411506 1.25922182
 1.61029525 1.82598664 2.03255503 1.93380262]


In [7]:
filename = 'data/stokes_shift_05.csv'
d = pd.read_csv(filename, sep=',',header=1,nrows=151)
# Drop the 'Unnamed: 4' column as it contains no useful data
d = d.drop(columns=['Unnamed: 4'])
ecc_lambda = d['Wavelength (nm).1'].dropna().to_numpy()
ecc_int = d['Intensity (a.u.).1'].dropna().to_numpy()


fig = go.Figure()

# Add excitation data
fig.add_trace(go.Scatter(x=ecc_lambda, y=ecc_int, mode='lines', name='Spettro di eccitazione'))

# Add peaks data
fig.add_trace(go.Scatter(x=LAMBDA_ECC, y=Picchi, mode='markers', name='Picchi'))

fig.update_layout(
    # title='Eccitazione vs Picchi',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Intensity (a.u.)',
    #legend_title='Legend',
    width=800,
    height=600,
    xaxis=dict(range=[440, 540]),
    legend=dict(
        x=0,
        y=1,
        xanchor='left',
        yanchor='top'
    )
)

fig.write_html(dir_path + '/html/spettro_emissione_rodamina_eccitata_a_diverse_frequenze_picchi.html')
fig.write_image(dir_path + '/images/spettro_emissione_rodamina_eccitata_a_diverse_frequenze_picchi.png')

fig.show()

In [13]:
# test di ipotesi

import scipy.interpolate as sc
print(Picchi, sigma_PICCHI)

spettro_eccitazione = sc.interp1d(ecc_lambda, ecc_int, kind='linear', fill_value='extrapolate')
χ2 = np.sum((Picchi - spettro_eccitazione(Picchi))**2 / sigma_PICCHI**2)
dof = len(Picchi) - 1

print(f'χ2: {χ2}')
print(f'dof: {dof}')

[110.0832379  163.88926662 280.30319838 393.06505053 466.18974343
 639.88802432 758.75275388 864.53429603 960.11318261 923.06582293] [0.23426658 0.33190078 0.59480244 0.787656   0.98411506 1.25922182
 1.61029525 1.82598664 2.03255503 1.93380262]
χ2: 156078813.3478532
dof: 9
